<a href="https://colab.research.google.com/github/SilvioIA2025/ALura2025/blob/main/Projeto_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
%pip -q install google-genai

In [230]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [233]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
from IPython.display import HTML, Markdown



In [234]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [235]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [262]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
import textwrap
from rich.markdown import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [263]:
##################################################################
# --- Agente 1: Buscador de Custos de Viagens Internacionais --- #
##################################################################
def agente_buscador(topico, pais, perfil, data_de_inicio, data_de_fim, numero_adultos, numero_crianças):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em calcular gastos em viagens internacionais, meu objetivo é fornecer uma estimativa detalhada e otimizada dos seus custos,
        considerando suas preferências e buscando as melhores opções em cada categoria.

        Instruções para o Especialista em Cálculos de Viagem:

        1. Compreensão do Perfil do Viajante:

        Destino: País(es) e Cidade(s).
        Período: Datas de partida e retorno (para considerar alta/baixa temporada e eventos).
        Número de Viajantes: Adultos e crianças.
        Estilo de Viagem: (e.g., mochileiro, econômico, confortável, luxuoso) para calibrar os valores.
        Interesses Específicos: (e.g., gastronomia, museus, aventura, compras) para alocar orçamento.

        2. Análise e Estimativa por Categoria:

        Voos (Melhores Opções):
        Pesquisar e comparar companhias aéreas, escalas, duração do voo e aeroportos de partida/chegada.
        Considerar voos diretos vs. com escalas (custo/benefício).
        Avaliar épocas de compra (antecedência) e flexibilidade de datas para melhores preços.
        Incluir custos de bagagem despachada, se aplicável.

        Aluguel de Carro:
        Avaliar a necessidade do aluguel versus transporte público no destino.
        Selecionar o tipo de veículo adequado (econômico, SUV, etc.).
        Incluir custos de seguro obrigatório e opcionais, combustível, pedágios e estacionamento.
        Comparar locadoras e políticas de retirada/devolução.

        Hotéis/Hospedagem (Melhores Opções):
        Sugerir tipos de hospedagem (hotéis, hostels, Airbnb, pousadas) com base no estilo de viagem.
        Considerar localização (proximidade a atrações, transporte), avaliações, comodidades (café da manhã, Wi-Fi) e política de cancelamento.
        Estimar custo por noite, ajustado pela temporada.

        Refeições:
        Estimar um custo médio diário por pessoa para café da manhã, almoço e jantar, diferenciando entre restaurantes econômicos, médios e sofisticados.
        Considerar o custo de lanches, bebidas e gorjetas.
        Perguntar se o viajante pretende cozinhar algumas refeições.

        Atrações Turísticas e Atividades:
        Listar as principais atrações pagas e o custo de seus ingressos.
        Sugerir passes turísticos (se vantajosos para a quantidade de atrações).
        Incluir custo de tours guiados, shows ou atividades específicas desejadas.

        Transporte Local:
        Custos de metrô, ônibus, bondes, táxis ou aplicativos de transporte (Uber, Lyft, etc.).
        Estimar passes diários/semanais se forem mais econômicos.
        Despesas Gerais e Extras:

        Seguro Viagem: Item obrigatório e essencial.
        Visto e Taxas de Entrada: Se aplicável ao destino.
        Comunicações: Custo de chip local ou roaming internacional.
        Emergências/Imprevistos: Sugerir uma reserva de 10-15% do total para gastos não planejados.
        Compras/Souvenirs: Definir um orçamento para compras pessoais e lembrancinhas.
        Outros: Lavanderia, gorjetas adicionais, pequenos gastos diversos.

        3. Saída Final:
        Faça voce a escolha de uma cidade a ser visitada no pais, caso o não tenha sido escolhida, retorne todos os resultados de custo pelo
        estilo de Viagem selecionado no perfil (mochileiro, economico, confortavel e luxuoso) e voce deve escolher automaticamente dois
        Interesses Específicos: dentre Gastronomia, Museus, Aventura e Compras.
        Apresentar um orçamento detalhado por categoria (Voos, Aluguel de Carro, Hotéis/Hospedagem, Refeições, Atrações Turísticas e Atividades, Transporte Local, Seguro Viagem
        Visto e Taxas de Entrada, Comunicações, Emergências/Imprevistos, Compras/Souvenirs e Outros ) , com uma estimativa total convertida para a moeda de origem do viajante
        (geralmente BRL) e da moeda local do destino, utilizando a taxa de câmbio atualizada. Oferecer dicas de economia e personalização.

        A sua tarefa é usar a ferramenta de busca do google (google_search) respeitando a data inicial e final da viagem
        Foque em no maximo 4 opções relevantes respeitando os estilos de Viagem: (mochileiro, econômico, confortável, luxuoso), com base no preço, categoria, e avaliaçoes dos clientes anteriores.
        Se uma avaliação tiver poucas informações ou reações entusiasmadas, é possivel que ela não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Essas avaliações relevantes devem ser atuais, de no maximo um tres meses da data de hoje.
        """,
        description = "Agente que busca melhores voos, carros, hoteis, refeiçoes, atrações turisticas, transporte e custos diretos e indiretos de viagem indicado no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nPaís: {pais}\nPerfil: {perfil}\nData de inicio: {data_de_inicio}\nData de fim: {data_de_fim}\nNumero_adultos: {numero_adultos}\nNumero_crianças: {numero_crianças}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [264]:
##########################################
# --- Agente 2: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, lancamentos_buscados):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em documentos profissionais.
            Por ter um público amplo, entre 18 e 70 anos, use sempre um tom de escrita adequado.
            Revise o lancamentos_buscados abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Separar por uma linha em branco cada item da lista.
            Se o lancamentos_buscados estiver bom, repita apenas o texto novamente'.
            Caso haja problemas, corrija e refaça o texto de maneira adequada, gerando uma versão corrigida
            """,
        description="Agente revisor de post para documentos profissionais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {lancamentos_buscados}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [265]:
##################################################################
# --- Agente 3: Buscador da Melhor alternativa de levar ME --- #
##################################################################
def agente_moeda(topico, pais):
    imagem = Agent(
        name="agente_moeda",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Voce como especialista em cambio da Viktap, por favor me faça um breve resumo de qual a melhor maneira para eu converter o meu
        dinheiro local para o dinheiro internacional, considerando cartao de credito, cartao de debito, cartao internacional, remessa para conta
        no estrangeiro, ou PIX. No caso do PIX, fale sobre o viktap consultando a fonte no site https://viktap.com.br/
        """,
        tools=[google_search],
    )
    entrada_do_agente_moeda = f"Tópico: {topico}\nPais: {pais}"
    # Executa o agente
    escolha_moeda = call_agent(imagem, entrada_do_agente_moeda)
    return escolha_moeda

In [286]:
####################################################################
# --- Agente 4: Sugestao de Voo, pelo perfil escolhido ----------- #
####################################################################
def agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno):
    viagem = Agent(
        name="agente_viagem",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais da LATAM, faça apenas uma opção resumida para um voo consultando a LATAM saindo
        do campo aeroporto e na data de ida data_inicio e data de retorno data_retorno
        por favor escolha um voo direto de ida e um voo direto de volta a sua escolha informando o numero do voo e horario de partida e chegada
        Caso a LATAM não possua voo para o destino, faça voce a escolha da companhia aerea, e se tambem nao conseguir voo direto pode escolher com escalas
        """,
        tools=[google_search],
    )
    entrada_do_agente_viagem = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nAeroporto: {aeroporto}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    viagem_pais = call_agent(viagem, entrada_do_agente_viagem)
    return viagem_pais

In [287]:
################################################################
# --- Agente 5: Sugestao de Hotel, refeição e passeio turistico
#     pelo perfil escolhido ---------------------------------- #
################################################################
def agente_outros(topico, pais, perfil, data_inicio, data_retorno):
    outros = Agent(
        name="agente_outros",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais, faça apenas uma opção resumida para hoteis, refeição e passeios turisticos
        levando em consideração na data de ida data_inicio e data de retorno data_retorno
        por favor escolha somente 3 opções de hoteis, passeios turisticos e refeicoes.
        """,
        tools=[google_search],
    )
    entrada_do_agente_outros = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    outros = call_agent(outros, entrada_do_agente_outros)
    return outros

In [291]:
######################################
# --- Agente 6: Redator do Post --- #
######################################
def agente_linkedin(topico, texto_revisado):
    linkedin = Agent(
        name="agente_linkedin",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Tervik, a empresa que desenvolveu o pagamento PIX com o Viktap .
            Utilize o tema fornecido no texto_revisado e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para o Linkedin sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final e mais a hashtag #imersaoiagoogle.
            """,
        description="Agente redator de posts engajadores para linkedin"
    )
    entrada_do_agente_linkedin = f"Tópico: {topico}\nPlano de post: {texto_revisado}"
    # Executa o agente
    linkedin = call_agent(linkedin, entrada_do_agente_linkedin)
    return linkedin

In [292]:
##agente_buscador(topico, pais, data_inicio, data_retorno, numero_adultos, numero_crianças)
##agente_revisor(topico, lancamentos_buscados)
##agente_moeda(topico, pais)
##agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
##agente_outros(topico, pais, perfil, data_inicio, data_retorno)
##agente_linkedin(topico, texto_revisado)

from datetime import datetime

def obter_data_do_usuario(param1):
    while True:
        if param1 == 'ida':
          data_str = input("Digite a data de inicio (DD/MM/AAAA): ")
        else:
          data_str = input("Digite a data de retorno (DD/MM/AAAA): ")

        try:
            # Tenta converter a string para um objeto datetime
            data_obj = datetime.strptime(data_str, "%d/%m/%Y")
            return data_obj
        except ValueError:
            print("Formato de data inválido. Por favor, use DD/MM/AAAA.")

def selecionar_perfil(opcoes, mensagem="Selecione uma opção:"):
    print(mensagem)
    for i, opcao in enumerate(opcoes):
        print(f"[{i + 1}] {opcao}")

    while True:
        try:
            escolha = input("Digite o número da sua escolha: ")
            indice_escolhido = int(escolha) - 1 # Ajusta para o índice da lista (0-based)

            if 0 <= indice_escolhido < len(opcoes):
                return opcoes[indice_escolhido]
            else:
                print("Número inválido. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")



In [293]:
topico = 'Viagem'
pais = input("Digite o Pais/Cidade do destino: ")
data_inicio = obter_data_do_usuario('ida')
data_retorno = obter_data_do_usuario('fim')
adultos = input("Digite o numero de adultos: ")
criancas = input("Digite o numero de crianças: ")
opcoes = ["Mochileiro", "Economico", "Confortavel", "Luxuoso"]
perfil = selecionar_perfil(opcoes, mensagem="Selecione uma opção:")
print (perfil)
aeroporto = input("Digite o aeroporto de partida: ")

lancamentos_buscados = agente_buscador(topico, pais, perfil, data_inicio, data_retorno, adultos, criancas)
##print("\n Resultado do Agente 1 (Buscador)! \n")
##display(to_markdown(lancamentos_buscados))
##print("----------------------------------------------------------------")

post_final = agente_revisor(topico, lancamentos_buscados)
display(to_markdown("-------------------------------"))
display(to_markdown("Recomendações para a sua viagem"))
display(to_markdown("-------------------------------"))
display(to_markdown(post_final))

escolha_moeda = agente_moeda(topico, pais)
display(to_markdown("--------------------------------------------------"))
display(to_markdown("Dicas financeiras para pagar com moeda estrangeira"))
display(to_markdown("--------------------------------------------------"))
display(to_markdown(escolha_moeda))

viagem_pais = agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
display(to_markdown("------------------------------------"))
display(to_markdown("Dica de um agente de viagem da LATAM"))
display(to_markdown("------------------------------------"))
display(to_markdown(viagem_pais))

outros = agente_outros(topico, pais, perfil, data_inicio, data_retorno)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Algumas dicas de Hoteis, refeiçoes e passeios turisticos"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(outros))

linkedin = agente_linkedin(topico, post_final)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Post do LinkedIn"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(linkedin))


Digite o Pais/Cidade do destino: Amsterdam
Digite a data de inicio (DD/MM/AAAA): 15/06/2025
Digite a data de retorno (DD/MM/AAAA): 30/06/2025
Digite o numero de adultos: 1
Digite o numero de crianças: 0
Selecione uma opção:
[1] Mochileiro
[2] Economico
[3] Confortavel
[4] Luxuoso
Digite o número da sua escolha: 4
Luxuoso
Digite o aeroporto de partida: GRU


▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Recomendações para a sua viagem                                                                                

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Compreendo. Como Editor e Revisor de Conteúdo, analisei o rascunho da estimativa de custos para a viagem de    
▌ luxo a Amsterdã. O texto está bem estruturado e contém as informações essenciais. No entanto, identifiquei     
▌ alguns pontos que podem ser aprimorados para garantir maior clareza, correção e polidez, especialmente em      
▌ relação à formatação de números e à precisão dos cálculos na tabela resumo.                                    
▌ Apresento a seguir a versão revisada e corrigida:                                                              
▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
▌ Olá! Como especialista em cálculos de viagens internacionais de luxo, estou pronto para ajudar a planejar sua  
▌ sofisticada estadia em Amsterdã, focando nos seus interesses em Gastronomia e Museus.                          
▌ Com base no seu perfil (Luxuoso, 1 adulto, de 15/06/2025 a 30/06/2025), preparei uma estimativa detalhada dos  
▌ custos, buscando as melhores opções para garantir conforto, exclusividade e experiências inesquecíveis.        
▌ Para a sua viagem de 15 dias em Amsterdã, na Holanda, no período de 15 a 30 de junho de 2025 (alta temporada), 
▌ com foco em luxo, gastronomia e museus, apresento a seguinte estimativa de gastos:                             
▌ Taxa de Câmbio Atual (Maio/2025):                                                                              
▌  • 1 EUR ≈ 6,31 BRL                                                                                            
▌  • 1 BRL ≈ 0,16 EUR                                                                                            
▌ Estimativa de Gastos (Valores em EUR e BRL):                                                                   
▌ É importante notar que os preços em Amsterdã, especialmente para hospedagem de luxo e refeições finas, podem   
▌ variar significativamente dependendo da antecedência da reserva e das escolhas específicas. Os valores em BRL  
▌ são baseados na taxa de câmbio informada e servem como referência.                                             
▌  • Voos (Classe Executiva/Primeira Classe): Pesquisas indicam voos em classe executiva de São Paulo para       
▌    Amsterdã com estimativas que se alinham ao perfil de luxo buscado. Considerando a busca por luxo, estimamos 
▌    um valor médio para ida e volta em Classe Executiva: Estimativa: € 3.500,00 - € 6.000,00 (R$ 22.085,00 - R$ 
▌    37.860,00)                                                                                                  
▌  • Aluguel de Carro: Em Amsterdã, o transporte público é eficiente e um carro pode não ser essencial, além de o
▌    estacionamento ser caro. Para o perfil luxuoso, serviços de transporte privado por aplicativo (como Uber    
▌    Black) ou motoristas particulares são mais adequados e convenientes. Estimativa: € 0,00 (Optando por        
▌    transporte privado por demanda)                                                                             
▌  • Hotéis/Hospedagem (Luxo): Hotéis 5 estrelas em Amsterdã têm diárias que podem variar bastante. Opções como  
▌    Waldorf Astoria Amsterdam (€ 1.200,00+/noite) e Pulitzer Amsterdam (€ 400,00+/noite) são exemplos. O        
▌    InterContinental Amstel Amsterdam também é uma opção de luxo com boa avaliação. Para 14 noites em um hotel  
▌    de luxo: Estimativa: € 6.000,00 - € 17.000,00 (R$ 37.860,00 - R$ 107.270,00) (Variável pela localização e   
▌    exclusividade)                                                                                              
▌  • Refeições (Foco em Gastronomia): Em restaurantes sofisticados, o custo do jantar pode variar entre € 30,00 e
▌    € 180,00 por pessoa. Considerando o foco em gastronomia de luxo, incluiremos jantares em restaurantes       
▌    renomados e almoços em locais de alta qualidade. O café da manhã pode ou não estar incluído na hospedagem.  
▌    Estim

...

▌ Para a sua viagem luxuosa a Amsterdã entre 15 e 30 de junho de 2025, aqui estão algumas opções resumidas de    
▌ hotéis, refeições e passeios turísticos:                                                                       
▌ Hotéis:                                                                                                        
▌  1 Rosewood Amsterdam: Com inauguração prevista para junho de 2025, este hotel promete ser uma opção de luxo de
▌    destaque, localizado no histórico distrito de canais.                                                       
▌  2 Waldorf Astoria Amsterdam: Situado em seis palácios do século XVII no cinturão de canais, Patrimônio Mundial
▌    da UNESCO, oferece acomodações elegantes com vista para o canal.                                            
▌  3 Sofitel Legend The Grand Amsterdam: Um hotel grandioso com uma rica história no coração de Amsterdã.        
▌ Refeições:                                                                                                     
▌  1 Ciel Bleu Restaurant: Conhecido por sua gastronomia europeia luxuosa.                                       
▌  2 Spectrum: Localizado no Waldorf Astoria, é um restaurante com duas estrelas Michelin que oferece uma jornada
▌    culinária multissensorial.                                                                                  
▌  3 Restaurant Rijks: Localizado ao lado do Rijksmuseum, destaca-se pela culinária holandesa refinada,          
▌    utilizando ingredientes locais.                                                                             
▌ Passeios Turísticos:                                                                                           
▌  1 Cruzeiro com jantar pelos canais: Uma experiência noturna atmosférica explorando os canais de Amsterdã com  
▌    um jantar de 4 tempos incluído.                                                                             
▌  2 Open Garden Days: Durante um fim de semana em junho, é possível visitar os deslumbrantes jardins privados de
▌    algumas das mais impressionantes casas nos canais.                                                          
▌  3 Holland Festival: O maior festival de artes cênicas da Holanda, ocorrendo durante a maior parte de junho,   
▌    com diversas apresentações de teatro, artes visuais, música clássica, dança, ópera e cinema.                

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Post do LinkedIn                                                                                               

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Okay, agente_linkedin, vamos transformar essa análise detalhada de custos de viagem de luxo em um post         
▌ engajador para o LinkedIn da Tervik, conectando o tema à nossa expertise em pagamentos.                        
▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
▌ Rascunho de Post para LinkedIn - Tervik                                                                        
▌ Sonhando com uma experiência internacional de luxo? ✈️ Imagine desfrutar do melhor da gastronomia e dos museus  
▌ em Amsterdã, sem se preocupar com as finanças.                                                                 
▌ Planejar uma viagem de alto padrão, como esta imersão de 15 dias em Amsterdã que detalhamos, vai muito além da 
▌ escolha dos destinos. Envolve mergulhar em estimativas de gastos complexas: voos em executiva, hotéis 5        
▌ estrelas, jantares em restaurantes renomados, ingressos para museus icônicos... e a constante atenção à taxa de
▌ câmbio (quem lida com Euro/Real sabe bem!).                                                                    
▌ Nosso exercício de planejamento mostrou que uma experiência de luxo como essa exige um orçamento considerável –
▌ e ter essa clareza é o primeiro passo.                                                                         
▌ Mas, depois de todo o planejamento, como garantir que a parte financeira seja tão fluida quanto a experiência  
▌ de viagem? 🤔                                                                                                  
▌ É aí que a tecnologia entra. Assim como o PIX revolucionou os pagamentos no Brasil pela sua simplicidade e     
▌ rapidez, ter soluções que trazem essa mesma facilidade para suas transações internacionais é fundamental.      
▌ Com a Tervik, estamos construindo o futuro dos pagamentos, tornando as transações internacionais mais simples, 
▌ eficientes e transparentes. Porque sua energia deve estar em aproveitar cada momento da sua viagem dos sonhos, 
▌ e não em burocracias de pagamento.                                                                             
▌ Planejamento financeiro detalhado + tecnologia de pagamento inteligente = Viagem de luxo impecável. ✨         
▌ #ViagemDeLuxo #PlanejamentoFinanceiro #PagamentosInternacionais #Fintech #ImersaoIAgoogle                      